In [ ]:
# Step 1: Import Libraries and Set Up Database Connection
# Purpose: Load necessary libraries, environment variables, and establish a connection to the PostgreSQL database.

import pandas as pd  # For data handling
import json  # To format JSON fields
from sqlalchemy import create_engine, text  # For database operations
from dotenv import load_dotenv  # To load environment variables
import os  # For interacting with the operating system

# Load environment variables from .env file
load_dotenv(r'C:\Users\Lane\Documents\Projects\trading_bot\programs\server_credentials.env')

# Retrieve database credentials from environment variables
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')

# Establish a connection to the PostgreSQL database
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
print("Database connection established.")


In [ ]:
# Step 2: Load the master_data CSV and Format Columns
# Purpose: Load the CSV file and format 'sector' and 'industry' columns as JSON arrays for database compatibility.

# Load master_data CSV file into a DataFrame
file_path = r'C:\Users\Lane\Documents\Projects\trading_bot\data\master_data12.csv'
new_data = pd.read_csv(file_path)

# Convert 'sector' and 'industry' columns to JSON arrays (with one element each) for database compatibility
new_data['sector'] = new_data['sector'].apply(lambda x: json.dumps([x]) if pd.notna(x) else json.dumps([]))
new_data['industry'] = new_data['industry'].apply(lambda x: json.dumps([x]) if pd.notna(x) else json.dumps([]))

print("Data loaded and formatted from master_data.")

In [ ]:
# Step 3: Load Existing Data from Database
# Purpose: Load existing data from the tracked_companies table to compare it with new_data and identify updates or new entries.

# Load data from the database table to compare with new_data
with engine.connect() as connection:
    existing_data = pd.read_sql(
        'SELECT symbol, asset_name, sector, industry, first_traded, index_inclusion FROM tracked_companies', 
        connection
    )
print("Existing data loaded from the database.")

New records successfully inserted into tracked_companies.


In [ ]:
# Step 4: Identify and Insert New Records
# Purpose: Insert any completely new symbols from master_data that aren't in the database.
#          This step ensures that any new stocks or companies in master_data get added.

# Identify records in new_data that are not already in the database
new_records = new_data[~new_data['symbol'].isin(existing_data['symbol'])]

# Insert these new records into the database
try:
    with engine.connect() as connection:
        new_records.to_sql('tracked_companies', con=connection, if_exists='append', index=False)
    print("New symbols and data successfully inserted into tracked_companies.")
except Exception as e:
    print(f"An error occurred during new record insertion: {e}")


In [ ]:
# Step 5: Identify Incomplete Records in Database
# Purpose: Identify records in the database that already exist (i.e., symbols already in tracked_companies)
#          but have NULL values in specific fields. These records need additional data from master_data.

with engine.connect() as connection:
    incomplete_records = pd.read_sql(
        """
        SELECT symbol, asset_name, sector, industry, first_traded, index_inclusion 
        FROM tracked_companies 
        WHERE asset_name IS NULL OR sector IS NULL OR industry IS NULL OR first_traded IS NULL OR index_inclusion IS NULL
        """,
        connection
    )
print("Incomplete symbols identified for updating.")

C:\Users\Lane\AppData\Local\Temp\ipykernel_16780\572457725.py:13: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if pd.isna(row[col]) or row[col] == '' or row[col] == '[]':


Updated CCJ with values: {'index_inclusion': '2016/12/01', 'symbol': 'CCJ'}
Existing records updated where needed.


In [ ]:
# Step 6: Update Incomplete Records with Data from master_data
# Purpose: Update missing fields for symbols in the database that already exist but have incomplete data.
#          Uses master_data to fill in only the missing fields, keeping the database as accurate as possible.

try:
    with engine.connect() as connection:
        for _, row in incomplete_records.iterrows():
            symbol = row['symbol']
            update_values = {}

            # Find the corresponding data in master_data for the symbol
            new_info = new_data[new_data['symbol'] == symbol]
            if not new_info.empty:
                # Check each field, and only add to update_values if the field in the database is null
                for col in ['asset_name', 'sector', 'industry', 'first_traded', 'index_inclusion']:
                    if pd.isna(row[col]) or row[col] == '' or row[col] == '[]':
                        if pd.notna(new_info.iloc[0][col]) and new_info.iloc[0][col] != '':
                            update_values[col] = new_info.iloc[0][col]

                # Execute the update statement if there are fields to update
                if update_values:
                    update_values['symbol'] = symbol  # Include symbol for WHERE clause
                    update_stmt = f"""
                    UPDATE tracked_companies
                    SET {', '.join([f"{col} = :{col}" for col in update_values.keys() if col != 'symbol'])}
                    WHERE symbol = :symbol
                    """
                    
                    # Execute the update with update_values as a dictionary
                    connection.execute(text(update_stmt), update_values)
                    print(f"Updated {symbol} with values: {update_values}")

    print("Existing symbols updated successfully.")
except Exception as e:
    print(f"An error occurred during record updates: {e}")


In [ ]:
# Step 7: Verification (Optional)
# Purpose: Verify updates by reloading data from the tracked_companies table to ensure that updates were applied correctly.
#          This is an optional step, helpful for validating the update process.

# Verify updates by reloading data from the tracked_companies table
with engine.connect() as connection:
    verified_data = pd.read_sql(
        'SELECT symbol, asset_name, sector, industry, first_traded, index_inclusion FROM tracked_companies', 
        connection
    )
print("Data verification complete. Updated data loaded for review.")
verified_data.head()
